In [ ]:
!pip install -U d2l

In [ ]:
!pip install skorch

In [3]:
%matplotlib inline
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch import nn
from torchsummary import summary
from d2l import torch as d2l
import numpy as np
from tensorflow import keras
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from skorch import NeuralNetClassifier

In [4]:
transform = transforms.Compose(
      [transforms.ToTensor(),
      transforms.Resize(224),
       transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

batch_size = 10

#validation and training set
full_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

#download 10000 testing dataset
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

#split 50k dataset into train and validation
train_size = int(0.7 * len(full_dataset))
validation_size = len(full_dataset) - train_size
train_dataset, validation_dataset = torch.utils.data.random_split(full_dataset, [train_size, validation_size])

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
def initialize_model(model_name, num_classes=10, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = torchvision.models.resnet18(pretrained=use_pretrained)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        nn.init.xavier_uniform_(model_ft.fc.weight);

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft

# KNN on original input

In [ ]:
#ref : https://towardsdatascience.com/multiclass-classification-using-k-nearest-neighbours-ca5281a9ef76

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

knn = KNeighborsClassifier()

#convert to 2d
x_train = np.reshape(x_train,(x_train.shape[0],-1))
x_test = np.reshape(x_test,(x_test.shape[0],-1))

knn.fit(x_train, y_train.ravel())

ypred = knn.predict(x_test)

acc = accuracy_score(y_test, ypred)

print(acc)

0.3398


# KNN on last layer

In [13]:
#softmax
#ref : implementation from tutorial 7 answer by Harrison
# https://www.youtube.com/watch?v=NVcJhMtRErE&list=PLyd8fQKbk5ej_lNdyAKI5aS_4hm-FhrrF&index=8
class NewModel(nn.Module):
  def __init__(self, pretrained_model, num_layers):
    super(NewModel, self).__init__()
    self.pretrained = pretrained_model
    self.flatten = nn.Flatten()
    #self.output = output_model
    #ref : https://forums.fast.ai/t/pytorch-best-way-to-get-at-intermediate-layers-in-vgg-and-resnet/5707
    self.features = nn.Sequential(*list(self.pretrained.children())[:num_layers])
  
#   def forward(self, x):
#     features = self.flatten.forward(self.features(x))
#     features.detach().numpy()
#     self.output.fit(features, y_train.ravel()) #classifier itself
#     return features

  def forward(self, x):
    x = x.permute(0, 3, 1, 2)
    features = self.flatten.forward(self.features(x))
    return self.output.forward(features)

In [15]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

#x_train = np.reshape(x_train,(x_train.shape[0],-1))
#x_test = np.reshape(x_test,(x_test.shape[0],-1))

x_train = x_train.float()

resnet = initialize_model('resnet')
model = NewModel(resnet, 8)

net = NeuralNetClassifier(
    model,
    max_epochs = 5,
    batch_size = 32,
    optimizer = torch.optim.SGD,
    iterator_train__shuffle = True,
    device = 'cuda',
    verbose = 1
)

#x_train = x_train.permute(0, 3, 1, 2)

output = net.fit(x_train, y_train)
print(output)

AttributeError: ignored